In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import bpy
import sys
import tempfile
import typst
from IPython.display import display, HTML
from pathlib import Path

project_root = Path.home() / "projects/blender_typst_importer/"
sys.path.append(str(project_root))


def display_svg(svg_content, width='100px'):
    html_content = (
        "<style>"
        ".svg-container svg { max-width: 100%; height: auto; display: block; }"
        "</style>"
    )
    html_content += f"<div class='svg-container' style='width:{width};'>{svg_content}</div>"
    display(HTML(html_content))

#uv pip install lxml
#uv pip install numpy==1.26.4
# so that code highlighting works

temp_dir = Path(tempfile.gettempdir())
#temp_dir = Path.cwd()
svg_file = temp_dir / "step1.svg"

In [3]:
file_content = """
#set page(width: 900pt, height: auto, margin: 0cm, fill: none)

#import "@preview/codelst:2.0.1": sourcecode
#import "@preview/tablex:0.0.8": tablex

#let sourcecode = sourcecode.with(frame: (code) => block(
  radius: 5pt,
  stroke: luma(30),
  inset: 30pt,
  text(size: 30pt, code)
))

#sourcecode[```python
42
```]
"""
typst_file = temp_dir / "step1.typ"
typst_file.write_text(file_content)

318

In [4]:
from typst_importer.typst_to_svg import simplify_svg, replace_stroke_with_path

typst.compile(typst_file, format="svg", output=str(svg_file))

step1_content = svg_file.read_text()
step2_content = simplify_svg(step1_content)
step3_content = replace_stroke_with_path(step2_content)

display_svg(step1_content , width='500px')
display_svg(step2_content , width='500px')
display_svg(step3_content , width='500px')

In [5]:
print(step2_content)

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:h5="http://www.w3.org/1999/xhtml" class="typst-doc" viewBox="0 0 900 97.79296875" width="900pt" height="97.79296875pt">
    <g>
        <g transform="translate(0 0)">
            <g class="typst-group">
                <g>
                    <g transform="translate(-0 -0)">
                        <path class="typst-shape" fill="none" stroke="#1e1e1e" stroke-width="1" stroke-linecap="butt" stroke-linejoin="miter" stroke-miterlimit="4" d="M 4.5 0 L 895.5 0 C 897.9853 0 900 2.0147185 900 4.5 L 900 93.29297 C 900 95.77825 897.9853 97.79297 895.5 97.79297 L 4.5 97.79297 C 2.0147185 97.79297 0 95.77825 0 93.29297 L 0 4.5 C 0 2.0147185 2.0147185 0 4.5 0 "/>
                    </g>
                    <g transform="translate(30 30)">
                        <g class="typst-group">
                            <g>
                                <g transform="translate(3 26.190234375)">
                  

In [6]:
from lxml import etree

svg = """
<?xml version="1.0" encoding="UTF-8"?>
<!-- Created on 2025-02-07 06:43:29 UTC by kolibril13 -->
<svg width="200" height="200" viewBox="0 0 200 200" xmlns="http://www.w3.org/2000/svg">
    <!-- Three non-overlapping strokes: one vertical, one horizontal, one angled -->
    <g transform="translate(0 0)">
        <!-- Vertical stroke -->
        <path class="stroke1" fill="none" 
              stroke="#1e1e1e" 
              stroke-width="2" 
              stroke-linecap="round" 
              stroke-linejoin="round"
              d="M 50,20 L 50,180" />
              
        <!-- Horizontal stroke -->
        <path class="stroke2" fill="none" 
              stroke="#1e1e1e" 
              stroke-width="2" 
              stroke-linecap="round" 
              stroke-linejoin="round"
              d="M 100,100 L 180,100" />
              
        <!-- Angled stroke -->
        <path class="stroke3" fill="none" 
              stroke="#1e1e1e" 
              stroke-width="2" 
              stroke-linecap="round" 
              stroke-linejoin="round"
              d="M 120,20 L 160,60" />
    </g>
</svg>
"""

def create_thick_line_path(x1, y1, x2, y2, thickness):
    """
    Create a path that represents a thick horizontal line as a rectangle.
    The line goes from (x1,y1) to (x2,y2) with given thickness.
    """
    half_thickness = thickness / 2
    path_d = (
        f"M {x1} {y1-half_thickness} "  # Start at top-left
        f"L {x2} {y2-half_thickness} "  # Line to top-right
        f"L {x2} {y2+half_thickness} "  # Line to bottom-right
        f"L {x1} {y1+half_thickness} "  # Line to bottom-left
        f"Z"
    )
    return path_d


def replace_stroke_with_path(svg_content):
    """Convert stroked paths to filled paths in the given SVG content."""
    parser = etree.XMLParser(remove_blank_text=True)
    svg_root = etree.fromstring(svg_content.strip(), parser)
    for path in svg_root.findall(".//*[@stroke-width]"):
        original_d = path.get("d")
        stroke_width = float(path.get("stroke-width"))
        parts = original_d.strip().split()
        if len(parts) >= 6 and parts[0] == "M" and parts[3] == "L":
            x1, y1 = float(parts[1]), float(parts[2])
            x2, y2 = float(parts[4]), float(parts[5])
            path.set("d", create_thick_line_path(x1, y1, x2, y2, stroke_width))
            path.set("fill", path.get("stroke", "#000000"))
            for attr in [
                "stroke",
                "stroke-width",
                "stroke-linecap",
                "stroke-linejoin",
                "stroke-miterlimit",
            ]:
                if attr in path.attrib:
                    del path.attrib[attr]
            if path.get("fill") == "none":
                path.set("fill", "#000000")
    return etree.tostring(svg_root, pretty_print=True, encoding="unicode") 

svg2 = replace_stroke_with_path(svg)

print(svg2)

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:h5="http://www.w3.org/1999/xhtml" class="typst-doc" viewBox="0 0 900 97.79296875" width="900pt" height="97.79296875pt">
  <g>
    <g transform="translate(0 0)">
      <g class="typst-group">
        <g>
          <g transform="translate(-0 -0)">
            <path class="typst-shape" fill="#1e1e1e" d="M 4.5 -0.5 L 895.5 -0.5 L 895.5 0.5 L 4.5 0.5 Z"/>
          </g>
          <g transform="translate(30 30)">
            <g class="typst-group">
              <g>
                <g transform="translate(3 26.190234375)">
                  <g class="typst-text" transform="scale(1, -1)">
                    <g fill="#a0a0a0" fill-rule="nonzero">
                      <path d="M 2.53125 1.5937501 L 5.4750004 1.5937501 L 5.4750004 12.290626 L 2.30625 11.578125 L 2.30625 13.303125 L 5.45625 13.996876 L 7.3500004 13.996876 L 7.3500004 1.5937501 L 10.25625 1.5937501 L 10.25625 0 L 2.53125 0 L 2.53125 1.593750

In [7]:
display_svg(svg, width='1000px')
display_svg(svg2, width='1000px')

In [8]:
# from gemini



from lxml import etree

svg = """
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink"
    xmlns:h5="http://www.w3.org/1999/xhtml" class="typst-doc" viewBox="0 0 900 97.79296875"
    width="900pt" height="97.79296875pt">
    <g>
        <g transform="translate(0 0)">
            <g class="typst-group">
                <g>
                    <g transform="translate(-0 -0)">
                        <path class="typst-shape" fill="none" stroke="#1e1e1e" stroke-width="1"
                            stroke-linecap="butt" stroke-linejoin="miter" stroke-miterlimit="4"
                            d="M 4.5 0 L 895.5 0 C 897.9853 0 900 2.0147185 900 4.5 L 900 93.29297 C 900 95.77825 897.9853 97.79297 895.5 97.79297 L 4.5 97.79297 C 2.0147185 97.79297 0 95.77825 0 93.29297 L 0 4.5 C 0 2.0147185 2.0147185 0 4.5 0 " />
                    </g>
                    <g transform="translate(30 30)">
                        <g class="typst-group">
                            <g>
                                <g transform="translate(3 26.190234375)">
                                    <g class="typst-text" transform="scale(1, -1)">
                                        <g fill="#a0a0a0" fill-rule="nonzero">
                                            <path
                                                d="M 2.53125 1.5937501 L 5.4750004 1.5937501 L 5.4750004 12.290626 L 2.30625 11.578125 L 2.30625 13.303125 L 5.45625 13.996876 L 7.3500004 13.996876 L 7.3500004 1.5937501 L 10.25625 1.5937501 L 10.25625 0 L 2.53125 0 L 2.53125 1.5937501 Z " />
                                        </g>
                                    </g>
                                </g>
                                <g transform="translate(24.559375 30.29296875)">
                                    <g class="typst-text" transform="scale(1, -1)">
                                        <g fill="#b60157" fill-rule="nonzero">
                                            <path
                                                d="M 10.766602 19.174805 L 3.8671875 7.6171875 L 10.766602 7.6171875 L 10.766602 19.174805 Z M 10.283203 21.870117 L 13.7109375 21.870117 L 13.7109375 7.6171875 L 16.625977 7.6171875 L 16.625977 5.2148438 L 13.7109375 5.2148438 L 13.7109375 0 L 10.766602 0 L 10.766602 5.2148438 L 1.4941406 5.2148438 L 1.4941406 8.012695 L 10.283203 21.870117 Z " />
                                        </g>
                                        <g transform="translate(18.0615234375,0.0)" fill="#b60157"
                                            fill-rule="nonzero">
                                            <path
                                                d="M 5.463867 2.4902344 L 15.512695 2.4902344 L 15.512695 0 L 2.2265625 0 L 2.2265625 2.4902344 Q 4.9658203 5.3759766 7.0166016 7.5878906 Q 9.067383 9.799805 9.84375 10.708008 Q 11.308594 12.495117 11.821289 13.601074 Q 12.333984 14.707031 12.333984 15.864258 Q 12.333984 17.695313 11.257324 18.735352 Q 10.180664 19.77539 8.305664 19.77539 Q 6.9726563 19.77539 5.5078125 19.291992 Q 4.0429688 18.808594 2.4023438 17.827148 L 2.4023438 20.81543 Q 3.9111328 21.533203 5.3686523 21.899414 Q 6.826172 22.265625 8.24707 22.265625 Q 11.455078 22.265625 13.410645 20.559082 Q 15.366211 18.85254 15.366211 16.083984 Q 15.366211 14.677734 14.714355 13.271484 Q 14.0625 11.865234 12.597656 10.166016 Q 11.777344 9.213867 10.217285 7.529297 Q 8.657227 5.8447266 5.463867 2.4902344 Z " />
                                        </g>
                                    </g>
                                </g>
                            </g>
                        </g>
                    </g>
                </g>
            </g>
        </g>
    </g>
</svg>
"""

def create_thick_line_path(x1, y1, x2, y2, thickness):
    """
    Create a path that represents a thick line as a rectangle.
    The line goes from (x1,y1) to (x2,y2) with given thickness.
    """
    half_thickness = thickness / 2
    if x1 == x2:  # Vertical line
        path_d = (
            f"M {x1-half_thickness} {y1} "  # Start at top-left
            f"L {x1+half_thickness} {y1} "  # Line to top-right
            f"L {x1+half_thickness} {y2} "  # Line to bottom-right
            f"L {x1-half_thickness} {y2} "  # Line to bottom-left
            f"Z"
        )
    else:  # Horizontal line
        path_d = (
            f"M {x1} {y1-half_thickness} "  # Start at top-left
            f"L {x2} {y2-half_thickness} "  # Line to top-right
            f"L {x2} {y2+half_thickness} "  # Line to bottom-right
            f"L {x1} {y1+half_thickness} "  # Line to bottom-left
            f"Z"
        )
    return path_d


def replace_stroke_with_path(svg_content):
    """Convert stroked paths to filled paths in the given SVG content."""
    parser = etree.XMLParser(remove_blank_text=True)
    svg_root = etree.fromstring(svg_content.strip(), parser)
    for path in svg_root.findall(".//*[@stroke-width]"):
        original_d = path.get("d")
        stroke_width = float(path.get("stroke-width"))
        parts = original_d.strip().split()
        if len(parts) >= 6 and parts[0] == "M" and parts[3] == "L":
            x1, y1 = float(parts[1]), float(parts[2])
            x2, y2 = float(parts[4]), float(parts[5])
            path.set("d", create_thick_line_path(x1, y1, x2, y2, stroke_width))
            path.set("fill", path.get("stroke", "#000000"))
            for attr in [
                "stroke",
                "stroke-width",
                "stroke-linecap",
                "stroke-linejoin",
                "stroke-miterlimit",
            ]:
                if attr in path.attrib:
                    del path.attrib[attr]
            if path.get("fill") == "none":
                path.set("fill", "#000000")
    return etree.tostring(svg_root, pretty_print=True, encoding="unicode") 

svg2 = replace_stroke_with_path(svg)

print(svg2)

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:h5="http://www.w3.org/1999/xhtml" class="typst-doc" viewBox="0 0 900 97.79296875" width="900pt" height="97.79296875pt">
  <g>
    <g transform="translate(0 0)">
      <g class="typst-group">
        <g>
          <g transform="translate(-0 -0)">
            <path class="typst-shape" fill="#1e1e1e" d="M 4.5 -0.5 L 895.5 -0.5 L 895.5 0.5 L 4.5 0.5 Z"/>
          </g>
          <g transform="translate(30 30)">
            <g class="typst-group">
              <g>
                <g transform="translate(3 26.190234375)">
                  <g class="typst-text" transform="scale(1, -1)">
                    <g fill="#a0a0a0" fill-rule="nonzero">
                      <path d="M 2.53125 1.5937501 L 5.4750004 1.5937501 L 5.4750004 12.290626 L 2.30625 11.578125 L 2.30625 13.303125 L 5.45625 13.996876 L 7.3500004 13.996876 L 7.3500004 1.5937501 L 10.25625 1.5937501 L 10.25625 0 L 2.53125 0 L 2.53125 1.593750